### Step 0: Import Libraries

In [2]:
import pandas as pd
import numpy as np

### Step 1: Load & Examine Data

Dataset Link: https://finrafoundation.org/sites/finrafoundation/files/2021-SxS-Data-and-Data-Info.zip

In [3]:
data = pd.read_csv('../data/NFCS 2021 State Data 220627.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27118 entries, 0 to 27117
Columns: 126 entries, NFCSID to wgt_s3
dtypes: float64(3), int64(83), object(40)
memory usage: 26.1+ MB


In [4]:
data.head()

,NFCSID,STATEQ,CENSUSDIV,CENSUSREG,A50A,A3Ar_w,A50B,A4A_new_w,A5_2015,A6,...,M6,M7,M8,M31,M50,M9,M10,wgt_n2,wgt_d2,wgt_s3
0,2021010001,41,5,3,2,2,8,1,6,1,...,1,3,2,2,2,1,2,0.834316,0.539386,0.725252
1,2021010002,36,3,2,2,2,8,1,6,4,...,98,98,98,98,98,1,98,1.083618,1.075806,0.930410
2,2021010003,3,8,4,1,6,6,1,6,4,...,1,3,98,3,1,1,2,0.396368,2.123406,0.944175
3,2021010004,3,8,4,2,4,10,2,2,1,...,1,98,98,1,1,1,2,0.374328,2.372112,1.011643
4,2021010005,36,3,2,2,4,10,1,3,4,...,1,98,98,98,98,98,98,1.362034,1.159651,0.907194


**Retaining only below columns**

|Column Name| Feature/Target | Values|
|:--------:|:--------:|:--------|
|A3Ar_w| Age group | <ul><li>1 18-24</li><li>2 25-34</li><li>3 35-44</li><li>4 45-54</li><li>5 55-64</li><li>6 65+</li></ul>|
|A5_2015| Education Level| <ul><li>1 Did not complete high school</li><li>2 High school graduate - regular high school diploma</li><li>3 High school graduate - GED or alternative credential</li><li>4 Some college, no degree</li><li>5 Associate's degree</li><li>6 Bachelor's degree</li><li>7 Post graduate degree</li><li>99 Prefer not to say</li></ul>|
|A8_2021| Annual Income| <ul><li>1 < $15,000</li><li>2 $15,000 - $25,000</li><li>3 $25,000 - $35,000</li><li>4 $35,000 - $50,000</li><li>5 $50,000 - $75,000</li><li>6 $75,000 - $100,000</li><li>7 $100,000 - $150,000</li><li>8 $150,000 - $200,000</li><li>9 $200,000 - $300,000</li><li>10 > $300,000</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|J5| Emergency Savings| <ul><li>1 Yes</li><li>2 No</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|J8| Retirement Planning| <ul><li>1 Yes</li><li>2 No</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|M7| Knoowledge: Interest Rate vs Inflation| <ul><li>1 More than today</li><li>2 Exactly the same</li><li>3 Less than today (Correct)</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|M8| Knowledge: Bonds vs Interest Rates|<ul><li>1 They will rise</li><li>2 They will fall (Correct)</li><li>3 They will stay the same </li><li>4 There is no relationship between bond prices and the interest</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|M10| Knowledge: Stocks vs Mutual Funds| <ul><li>1 True</li><li>2 False</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|
|J2| Risk Tolerance| <ul><li>1 Not At All Willing</li><li>2 </li><li>3 </li><li>4 </li><li>5 </li><li>6 </li><li>7 </li><li>8 </li><li>9 </li><li>10 Very Willing</li><li>98 Don't know</li><li>99 Prefer not to say</li></ul>|


### Step 2: Retain important features only

In [5]:

required_columns = ['A3Ar_w', 'A5_2015', 'A8_2021', 'J5', 'J8', 'M7', 'M8', 'M10', 'J2']
data = data[required_columns]
data = data.dropna(subset=required_columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27118 entries, 0 to 27117
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   A3Ar_w   27118 non-null  int64 
 1   A5_2015  27118 non-null  int64 
 2   A8_2021  27118 non-null  int64 
 3   J5       27118 non-null  int64 
 4   J8       27118 non-null  object
 5   M7       27118 non-null  int64 
 6   M8       27118 non-null  int64 
 7   M10      27118 non-null  int64 
 8   J2       27118 non-null  int64 
dtypes: int64(8), object(1)
memory usage: 1.9+ MB


In [6]:
data.head()

,A3Ar_w,A5_2015,A8_2021,J5,J8,M7,M8,M10,J2
0,2,6,9,1,1,3,2,2,9
1,2,6,3,2,2,98,98,98,1
2,6,6,3,1,,3,98,2,7
3,4,2,7,1,1,98,98,2,7
4,4,3,1,2,2,98,98,98,5


### Step 3: Calculate financial literacy score based on M7, M8 and M10

In [7]:

# Ensure columns are numeric
data[['M7', 'M8', 'M10']] = data[['M7', 'M8', 'M10']].apply(pd.to_numeric, errors='coerce')

# Define correct answers
correct_answers = {
    'M7': 3,
    'M8': 2,
    'M10': 2
}

# Function to assign 1 for correct answer, 0 otherwise (including 98/99/missing)
def is_correct(value, correct_option):
    return 1 if value == correct_option else 0

# Apply scoring
data['M7_correct'] = data['M7'].apply(lambda x: is_correct(x, correct_answers['M7']))
data['M8_correct'] = data['M8'].apply(lambda x: is_correct(x, correct_answers['M8']))
data['M10_correct'] = data['M10'].apply(lambda x: is_correct(x, correct_answers['M10']))

# Compute financial literacy score
data['financial_literacy_score'] = data['M7_correct'] + data['M8_correct'] + data['M10_correct']

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27118 entries, 0 to 27117
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   A3Ar_w                    27118 non-null  int64 
 1   A5_2015                   27118 non-null  int64 
 2   A8_2021                   27118 non-null  int64 
 3   J5                        27118 non-null  int64 
 4   J8                        27118 non-null  object
 5   M7                        27118 non-null  int64 
 6   M8                        27118 non-null  int64 
 7   M10                       27118 non-null  int64 
 8   J2                        27118 non-null  int64 
 9   M7_correct                27118 non-null  int64 
 10  M8_correct                27118 non-null  int64 
 11  M10_correct               27118 non-null  int64 
 12  financial_literacy_score  27118 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 2.7+ MB


### Step 4: Rename columns

In [9]:

data = data.rename(columns={
    'A3Ar_w': 'age_group',
    'A5_2015': 'education_level',
    'A8_2021': 'income_bracket',
    'J5': 'emergency_savings',
    'J8': 'retirement_planning',
    'J2': 'risk_tolerance'
})[[
    'age_group', 'education_level', 'income_bracket',
    'emergency_savings', 'retirement_planning',
    'financial_literacy_score', 'risk_tolerance'
]]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27118 entries, 0 to 27117
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   age_group                 27118 non-null  int64 
 1   education_level           27118 non-null  int64 
 2   income_bracket            27118 non-null  int64 
 3   emergency_savings         27118 non-null  int64 
 4   retirement_planning       27118 non-null  object
 5   financial_literacy_score  27118 non-null  int64 
 6   risk_tolerance            27118 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 1.4+ MB


### Step 5: Handle missing values

In [11]:
data.isnull().sum()

age_group                   0
education_level             0
income_bracket              0
emergency_savings           0
retirement_planning         0
financial_literacy_score    0
risk_tolerance              0
dtype: int64

In [12]:
for col in data.columns:
    print(data[col].value_counts())
    print("Total:", data[col].count())
    print("--------")

age_group
6    5501
5    4731
2    4696
4    4617
3    4564
1    3009
Name: count, dtype: int64
Total: 27118
--------
education_level
4    7065
6    6682
2    4825
7    2969
5    2943
3    1897
1     737
Name: count, dtype: int64
Total: 27118
--------
income_bracket
5     5007
4     3847
6     3570
7     3470
1     3327
2     2942
3     2918
8     1212
9      560
10     265
Name: count, dtype: int64
Total: 27118
--------
emergency_savings
1     14493
2     11436
98      844
99      345
Name: count, dtype: int64
Total: 27118
--------
retirement_planning
2     11491
1      8418
       6058
98      916
99      235
Name: count, dtype: int64
Total: 27118
--------
financial_literacy_score
0    8381
1    7860
2    6936
3    3941
Name: count, dtype: int64
Total: 27118
--------
risk_tolerance
5     3865
1     3809
7     3354
6     3110
3     2641
4     2432
8     2269
2     2021
10    2000
9      957
98     538
99     122
Name: count, dtype: int64
Total: 27118
--------


In [13]:
# Identify blank spaces and convert to NaN
for col in data.columns:
    if data[col].isnull().sum() > 0:
        print(f"{col} has {data[col].isnull().sum()} null values")
    if (data[col] == '').any():
        print(f"{col} has {(data[col] == '').sum()} empty strings")
        data[col].replace('', np.nan)
    if (data[col].astype(str).str.isspace()).any():
        print(f"{col} has {data[col].astype(str).str.isspace().sum()} blank spaces")
        data[col] = data[col].apply(lambda x: x.strip()).replace('', -1) # -1 to indicate missing value



retirement_planning has 6058 blank spaces


In [14]:
data['retirement_planning'] = data['retirement_planning'].astype(int)

In [15]:
# Convert all 98 and 99 values to -1 indicating missing value
data = data.replace(98, -1)
data = data.replace(99, -1)

In [16]:
for col in data.columns:
    print(f"{data[col].value_counts(normalize=True) * 100}")
    # print(data[data[col] == -1].count() / data.count() * 100)
    print("--------")

age_group
6    20.285419
5    17.445977
2    17.316911
4    17.025592
3    16.830150
1    11.095951
Name: proportion, dtype: float64
--------
education_level
4    26.052806
6    24.640460
2    17.792610
7    10.948448
5    10.852570
3     6.995354
1     2.717752
Name: proportion, dtype: float64
--------
income_bracket
5     18.463751
4     14.186149
6     13.164688
7     12.795929
1     12.268604
2     10.848883
3     10.760381
8      4.469356
9      2.065049
10     0.977211
Name: proportion, dtype: float64
--------
emergency_savings
 1    53.444207
 2    42.171252
-1     4.384542
Name: proportion, dtype: float64
--------
retirement_planning
 2    42.374069
 1    31.042112
-1    26.583819
Name: proportion, dtype: float64
--------
financial_literacy_score
0    30.905672
1    28.984438
2    25.577107
3    14.532783
Name: proportion, dtype: float64
--------
risk_tolerance
 5     14.252526
 1     14.046021
 7     12.368169
 6     11.468397
 3      9.738919
 4      8.968213
 8      8.367136

Since risk_tolerance is our target variable, we will drop the 2.43% data (-1 values)

In [17]:
data = data[data['risk_tolerance'] != -1]
len(data)

26458

Since emergency_savings has 4.4% missing values, we will treat them as 'No'

In [18]:
data['emergency_savings'] = data['emergency_savings'].replace(-1, 2)

We have ~26.6% missing data for retirement_planning, we will treat it as a third category 'Unknown' using value as 3

In [19]:
data['retirement_planning'] = data['retirement_planning'].replace(-1, 3)

### Step 6: Save cleaned data

In [ ]:
data.to_parquet('../data/finra_cleaned.parquet', index=False)
print("✅ Cleaned FINRA dataset saved to data/finra_cleaned.csv")